# Dynamic Programming

This notebook assembles the advanced dynamic programing problem to helpe summerize the pattern of solution same familiar problem.

# Coin Change II

You are given coins of different denominations and a total amount of money. Write a function to compute the number of combinations that make up that amount. You may assume that you have infinite number of each kind of coin.

**Example 1:**
```py
Input: amount = 5, coins = [1, 2, 5]
Output: 4
Explanation: there are four ways to make up the amount:
5=5
5=2+2+1
5=2+1+1+1
5=1+1+1+1+1
```
**Example 2:**
```py
Input: amount = 3, coins = [2]
Output: 0
Explanation: the amount of 3 cannot be made up just with coins of 2.
```
**Example 3:**
```py
Input: amount = 10, coins = [10] 
Output: 1
```

As the steps using in the Dynamic programming I, when we encounter those dynamic programming, the first step is to determine the recursion relation of the the subproblems. The entrtpiont for us to come up with our subproblem is the arguments on the example. In common, the arguments determine the memorize cahe dimension we need.

The coin change II has two arguments that used to describe the problem. So we can easy think of our subproblem from the arguments. Create a table for our subproblems, the row use the `coins set` argument, and the columns use our `ammount` argument. (I use the example 1 as our specifial condition for find the subproblem realtionship.

coins/amount | 0 | 1 | 2 | 3 | 4 | 5 
--|--|--|--|--|--|--
[]|1|0|0|0|0|0
[1]|1|1|1|1|1|1
[1,2]|1|1|2

In [69]:
def naive_make_change(amount, coins):
    # base case is the amount == 0 and coins set is empty
    if amount == 0:
        return 1
    if not coins:
        return 0
    
    if amount < coins[0]:
        # if amount is bigger than our the first denomination on our coins set,
        # we need to skip it as it can not help us to change our amount.
        return naive_make_change(amount, coins[1:])
    
    # when the first coin on our coins set can be use to change,
    # we have two choice to do:
    # 1. not use the first coin, skip it use the remain coins set to change our amount
    # 2. use the first coin, the amount decrease by its denomination. and for us can
    # use the first coin again, not change our coin sets.
    return naive_make_change(amount, coins[1:]) + naive_make_change(amount-coins[0], coins)

In [70]:
naive_make_change(5, [1, 2, 5])

4

In [71]:
import timeit
timeit.timeit('naive_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.007942766998894513

In [72]:
def memo_make_change(amount, coins, memo={}):
    if amount == 0:
        return 1
    if not coins:
        return 0
    
    if amount < coins[0]:
        return memo_make_change(amount, coins[1:], memo)
    
    if (amount, coins[0]) in memo:
        return memo[(amount, coins[0])]
    ans = memo_make_change(amount, coins[1:], memo) + memo_make_change(amount-coins[0], coins, memo)
    memo[(amount, coins[0])] = ans
    return ans

In [73]:
memo_make_change(5, [1, 2, 5])

4

In [74]:
timeit.timeit('memo_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.0004568040603771806

In [47]:
def bottom_up_make_change(amount, coins):
    # Creat the table, row is sub coins set. col is our amount.
    # Our row length will go to amount + 1, as we will conside the situration
    # that amount is 0, when no matter what our coins set going to be.
    # the our result will always be 1. Means change amount 0, there at least have 1
    # possible to get change.
    dp = [[1 if i == 0 else 0 for i in range(amount+1)] for _ in range(len(coins))]
    for a in range(1, amount+1):
        if coins[0] <= a:
            dp[0][a] = dp[0][a-coins[0]]
    
    for coin_row in range(1, len(coins)):
        for a in range(1, amount+1):
            if coins[coin_row] <= a:
                # Our coin can be use to change our amount `a`.
                dp[coin_row][a] = dp[coin_row-1][a] + dp[coin_row][a - coins[coin_row]]
            else:
                # Our current coin can not use to change our amount `a`.
                dp[coin_row][a] = dp[coin_row-1][a]
    return dp[-1][-1]

In [48]:
bottom_up_make_change(5, [1, 2, 5])

4

In [49]:
timeit.timeit('bottom_up_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.00885312701575458

In [81]:
def bottom_up_make_change_two_row(amount, coins):
    if amount == 0:
        return 1
    if not coins:
        return 0
    pre_row = [1 if i == 0 else 0 for i in range(amount+1)]
    for a in range(1, amount+1):
        if coins[0] <= a:
            pre_row[a] = pre_row[a-coins[0]]

    if len(coins) == 1:
        return pre_row[-1]
    for coin_id in range(1, len(coins)):
        cur = [1]
        for a in range(1, amount+1):
            if coins[coin_id] <= a:
                cur.append(pre_row[a] + cur[a-coins[coin_id]])
            else:
                cur.append(pre_row[a])
        pre_row = cur
    return cur[-1]

In [82]:
bottom_up_make_change_one_row(5, [1, 2, 5])

4

In [83]:
timeit.timeit('bottom_up_make_change_one_row(5, [1, 2, 5])', number=1000, globals=globals())

0.006271212012507021

In [88]:
def bottom_up_make_change_one_row(amount, coins):
    if amount == 0:
        return 1
    if not coins:
        return 0
    dp = [1 if i == 0 else 0 for i in range(amount+1)]
    for coin_id in range(len(coins)):
        for a in range(coins[coin_id], amount+1):
            dp[a] += dp[a - coins[coin_id]]
        print(dp)
    return dp[-1]

In [89]:
bottom_up_make_change_one_row(5, [1, 2, 5])

[1, 1, 1, 1, 1, 1]
[1, 1, 2, 2, 3, 3]
[1, 1, 2, 2, 3, 4]


4

In [140]:
def numDecodings(s):
    if not s:
        return 1
    if int(s[0]) == 0:
        return 0
    if len(s) > 1 and int(s[0:2]) <= 26:
        return numDecodings(s[1:]) + numDecodings(s[2:])
    
    else:
        return numDecodings(s[1:])

In [173]:
numDecodings("101")

1

In [169]:
def memo_numDecodings(s, memo={}):
    if not s:
        return 1
    if int(s[0]) == 0:
        return 0
    if s in memo:
        return memo[s]
    if len(s) > 1 and int(s[0:2]) <= 26:
        ans = numDecodings(s[1:]) + numDecodings(s[2:])
        memo[s] = ans
        return ans
    else:
        ans = numDecodings(s[1:])
        memo[s] = ans
        return ans

In [170]:
memo_numDecodings("1201020112")

3

In [171]:
timeit.timeit('memo_numDecodings("1123123123")', number=1000, globals=globals())

0.0005149770295247436

In [190]:
def bottom_up_numDecodings(s):
    if not s:
        return 1
    s = s[::-1]
    dp = [0 for _ in range(len(s)+1)]
    dp[0] = 1
    for i in range(1, len(s) + 1):
        if i == 1:
            dp[i] = 1 if s[i-1] != "0" else 0
        elif s[i-1] == "0":
            dp[i] = 0
        elif int(s[i-1] + s[i-2]) <= 26:
            dp[i] = dp[i-1] + dp[i-2]
        else:
            dp[i] = dp[i-1]
    return dp[-1]

In [191]:
bottom_up_numDecodings("120")

1

In [192]:
timeit.timeit('bottom_up_numDecodings("1123123123")', number=1000, globals=globals())

0.007856849930249155

In [193]:
def bottom_up_numDecodings_one_row(s):
    if not s:
        return 1
    # Initial the varible, is mean when the str(s)
    pre = 1
    cur = 0
    for i in range(len(s)-1, -1, -1):
        if i == len(s)-1:
            cur = 1 if s[i] != "0" else 0
        elif s[i] == "0":
            pre = cur
            cur = 0
        elif int(s[i] + s[i+1]) <= 26:
            pre, cur = cur, cur + pre
        else:
            pre = cur
    return cur

In [194]:
timeit.timeit('bottom_up_numDecodings_one_row("1123123123")', number=1000, globals=globals())

0.004777148016728461

In [263]:
def countNumberBits(num):
    dp = [0 for _ in range(num+1)]
    dp[1] = 1
    for i in range(1, num + 1):
        dp[i] = i % 2 + dp[i // 2]
    return dp